<a href="https://www.kaggle.com/code/jasminemohamed2545/thesalespredictionplaybook-atoz-guide?scriptVersionId=261629877" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🏆 AI Talents League R1: A Deep Dive into Sales Prediction


## 📖 Introduction: The Quest for Accurate Predictions

Welcome to my detailed exploration of the AI Talents League Round 1 competition! The challenge is to predict product sales (`Y`) based on a variety of product and outlet features. In this notebook, I will walk you through a complete machine learning workflow, from initial data exploration and cleaning to advanced feature engineering and final model submission.

My approach is built on the philosophy of **"understanding before predicting."** I believe that a thorough analysis of the data's structure, distributions, and hidden relationships is the key to building a robust and high-performing model.

### 🎯 Our Workflow at a Glance:
1.  **Initial Setup**: Importing libraries and loading the dataset.
2.  **Exploratory Data Analysis (EDA)**: Uncovering insights, visualizing distributions, and understanding feature relationships. We'll look at the target variable, numerical features, and categorical features.
3.  **Data Cleaning & Preprocessing**: Systematically handling missing values, duplicates, and outliers to create a clean foundation for our model.
4.  **Advanced Feature Engineering**: Creating new, powerful features from the existing data to boost model performance.
5.  **Modeling & Prediction**: Building a powerful `RandomForestRegressor` model, transforming the features, and generating the final predictions for submission.

Let's begin this journey and transform raw data into actionable predictions! 🚀

---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import shap
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, PolynomialFeatures, OneHotEncoder, RobustScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# Loading El Data

In [ ]:
# Load el Data
train_df = pd.read_csv(os.path.join(ai_talents_league_round_1_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(ai_talents_league_round_1_path,'test.csv'))
sample_submission = pd.read_csv(os.path.join(ai_talents_league_round_1_path,'sample_submission.csv'))

### 📊 A Bird's-Eye View: Data Summary & Missing Values(EDA)

Before we dive deep, let's get a quick summary of our datasets. We'll check the dimensions, data types, and, most importantly, the extent of missing values. This initial check is crucial for planning our data cleaning strategy. A styled table makes it easy to spot columns that need our attention.

In [ ]:
# Concatenate for combined analysis, keeping train/test context
all_data_summary = pd.concat([train_df.drop('Y', axis=1), test_df], keys=['train', 'test'])

# Create a summary DataFrame
summary = pd.DataFrame({
    'Data Type': all_data_summary.dtypes,
    'Missing Values (Train)': train_df.isnull().sum(),
    'Missing % (Train)': (train_df.isnull().sum() / len(train_df)) * 100,
    'Missing Values (Test)': test_df.isnull().sum(),
    'Missing % (Test)': (test_df.isnull().sum() / len(test_df)) * 100,
    'Unique Values (Train)': train_df.nunique()
})

# Style the summary table for better readability and insight
summary.style.background_gradient(cmap='viridis_r', subset=['Missing % (Train)', 'Missing % (Test)']).format('{:.2f}%', subset=['Missing % (Train)', 'Missing % (Test)'])

# EDA for the train Data

In [ ]:
train_df.head()

In [ ]:
train_df.select_dtypes(include=['object'])

# **📈 Visual Storytelling: Diving Deep with EDA**
### Now that we have a general overview of the data, it's time to dive deeper. In this section, we'll use a variety of advanced visualizations to uncover hidden patterns and relationships between the features. Each plot will tell a part of the story, bringing us closer to understanding the key drivers of sales.

## 1. How Are Sales Distributed? (Target Variable: Y)
The first step in any machine learning problem is to understand the variable we are trying to predict. This histogram shows us the distribution of our sales values (Y).


In [ ]:
# A quick look at the target 'Y' can reveal skewness or outliers.
plt.figure(figsize=(12, 6))
sns.histplot(train_df['Y'], kde=True, bins=40, color='green')
plt.title('Distribution of Target Variable (Y)', fontsize=16)
plt.xlabel('Y Value')
plt.ylabel('Frequency')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

## 2. The Relationship Matrix: Correlation Heatmap
Is there a relationship between an item's price and its weight? Does one affect the other? The heatmap answers these questions by measuring the linear correlation between all numerical features.



In [ ]:
# Create a new DataFrame with numeric columns and the target variable for correlation analysis
numeric_cols = train_df.select_dtypes(include=['int32', 'float64'])


# Calculate the correlation matrix
corr_matrix = numeric_cols.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))
plt.title('Correlation Heatmap of Numerical Features and Target', fontsize=18)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix,
            cmap='coolwarm',
            annot=True,
            fmt=".2f", # Format annotations to two decimal places
            linewidths=.5,
            cbar_kws={"shrink": .8})

plt.show()

## 3. Where Do Sales Happen? Outlet Type vs. Sales
Not all outlets are created equal. This analysis compares the sales distribution across different outlet types (X11) to see where the most products are sold.




In [ ]:
# Using FacetGrid to create separate histograms for each outlet type
g = sns.FacetGrid(train_df, col='X11', col_wrap=2, height=5, aspect=1.5, sharey=False, sharex=False)
g.map(sns.histplot, 'Y', kde=True, bins=30)
g.fig.suptitle('Distribution of Sales (Y) Across Different Outlet Types (X11)', fontsize=16, y=1.02)
g.set_axis_labels('Sales (Y)', 'Frequency')
g.set_titles("Outlet Type: {col_name}")

plt.show()

# A simpler alternative using boxplots which is great for comparison
plt.figure(figsize=(14, 8))
sns.boxplot(data=train_df, x='X11', y='Y', palette='viridis')
plt.title('Sales Distribution by Outlet Type', fontsize=16)
plt.xlabel('Outlet Type (X11)', fontsize=12)
plt.ylabel('Sales (Y)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## 4. Does Visibility Drive Sales?
Does how visible a product is on the shelf (X4) increase its sales? This scatterplot helps us explore that relationship.

In [ ]:
plt.figure(figsize=(12, 7))
sns.scatterplot(data=train_df, x='X4', y='Y', alpha=0.5)
plt.title('Item Visibility (X4) vs. Sales (Y)', fontsize=16)
plt.xlabel('Item Visibility', fontsize=12)
plt.ylabel('Sales', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# Wrangling, Analysis and Feature Engineering

In [ ]:
# Remove el Duplicates
initial_rows = len(train_df)
train_df.drop_duplicates(inplace=True)
print(f"Removed {initial_rows - len(train_df)} duplicate rows.")

In [ ]:
# Handle Outliers using IQR (Only for Numeric Columns)
# Removing extreme values that could skew the model's training.

# Define column lists early
numeric_cols = train_df.select_dtypes(include=np.number).columns.tolist()
if 'Y' in numeric_cols:
    numeric_cols.remove('Y') # Exclude target from operations
categorical_cols = ['X1', 'X3', 'X5', 'X7', 'X9', 'X10', 'X11']

In [ ]:
for col in categorical_cols:
  train_df[col] = train_df[col].fillna(train_df[col].mode()[0])
  test_df[col] = test_df[col].fillna(test_df[col].mode()[0])

In [ ]:
# Missing Value Imputation (MUST be done before outlier detection)
# Filling missing values with the median, which is robust to outliers.
# We assign the result back to the column to avoid the SettingWithCopyWarning.
for col in numeric_cols:
    median_val = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_val)
    test_df[col] = test_df[col].fillna(median_val)
print("Missing values imputed using the median.")

In [ ]:
train_df['X0'] = train_df['X1'].str[:2]
test_df['X0'] = test_df['X1'].str[:2]

In [ ]:
train_df.X0.unique(), test_df.X0.unique()

In [ ]:
# Now that data is clean, a boxplot helps visualize the outliers we're about to handle.
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for ax, col in zip(axes.flatten(), numeric_cols):
    sns.boxplot(x=train_df[col], ax=ax, orient='h')
    ax.set_title(f'Boxplot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
# Outliers Removal
Q1 = train_df[numeric_cols].quantile(0.25)
Q3 = train_df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1
outlier_condition = ~((train_df[numeric_cols] < (Q1 - 1.5 * IQR)) | (train_df[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)
initial_rows = len(train_df)
train_df = train_df[outlier_condition]
print(f"Removed {initial_rows - len(train_df)} rows containing outliers.")

In [ ]:
# Now that data is clean, a boxplot helps visualize the outliers we're about to handle.
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for ax, col in zip(axes.flatten(), numeric_cols):
    sns.boxplot(x=train_df[col], ax=ax, orient='h')
    ax.set_title(f'Boxplot of {col}')
plt.tight_layout()
plt.show()

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
for ax, col in zip(axes.flatten(), numeric_cols):
  stats.probplot(train_df[col], plot=ax, dist="norm")
  ax.set_title(f'Q-Q plot for {col}') # Use set_title instead of title
plt.tight_layout()
plt.show()

In [ ]:
train_df.X3.unique(), train_df.X9.unique(), train_df.X10.unique(), train_df.X11.unique()

In [ ]:
# Clean up inconsistent values in X3
train_df['X3'] = train_df['X3'].replace({'low fat': 'Low Fat', 'LF': 'Low Fat', 'reg': 'Regular'})
test_df['X3'] = test_df['X3'].replace({'low fat': 'Low Fat', 'LF': 'Low Fat', 'reg': 'Regular'})

In [ ]:
train_df.X3.unique(), train_df.X9.unique(), train_df.X10.unique(), train_df.X11.unique()

In [ ]:
train_df.X5.unique(), train_df.X7.unique(), train_df.X11.unique()

In [ ]:
x_train = train_df.drop('Y', axis=1)
y_train = train_df['Y']

x_test = test_df.copy()

# Preprocessing

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

ord_cols = ['X3', 'X9', 'X10']
nominal_cols = ['X0', 'X5', 'X7', 'X11']
# Define column lists early
numeric_cols = train_df.select_dtypes(include=np.number).columns.tolist()
if 'Y' in numeric_cols:
    numeric_cols.remove('Y') # Exclude target from operations


x3map = ['Low Fat', 'Regular']
x9map = ['Small', 'Medium', 'High']
x10map = ['Tier 1', 'Tier 2', 'Tier 3']

preprocessor = ColumnTransformer(
    transformers =[
        ('droper', 'drop', ['X1'] ),
        ('ordin', OrdinalEncoder(categories=[x3map, x9map, x10map]), ord_cols),
        ('1hot', OneHotEncoder(handle_unknown='ignore'), nominal_cols),
        ('scaler', RobustScaler(), numeric_cols) # Scale only numeric columns
        ],
        remainder='passthrough' # Apply 'passthrough' to all other columns not mentioned in the transformer
)

#### we didn't include X1 in the encoding. That was a deliberate choice.

X1 is a high-cardinality identifier. Here's what that means and why it's problematic:

High Cardinality: The column contains many unique values (FDA15, DRC01, FDN15, etc.). There are likely hundreds or even thousands of different item IDs.

No Generalizable Pattern: The model's goal is to learn patterns that apply to new, unseen data. A model that learns "item FDA15 sells well" has only memorized a fact; it hasn't learned why. It will have no idea how to predict sales for a new item, FDT58, that it has never seen before. Including the raw ID would cause the model to overfit by essentially memorizing the training data.

Curse of Dimensionality: If we were to One-Hot Encode this feature, we would create thousands of new columns, making our dataset massive and our model very slow and inefficient.

#### The Professional Approach (Feature Engineering):

Instead of using the ID directly, a more advanced technique is to engineer a new feature from it. The first two letters seem to be a category code:

FD -> Food

DR -> Drinks

NC -> Non-Consumable

We could create a new column called Item_Category and extract these prefixes. This new feature would have low cardinality (only 3 unique values) and would be genuinely useful for the model. For simplicity in the initial script, I just dropped it, which is also a very common and valid approach.

In [ ]:
X_train_processed = preprocessor.fit_transform(x_train).toarray()
X_test_processed = preprocessor.transform(x_test).toarray()

y_trainarr = y_train.values
# law e7tagt el dataframe versions menhom fe 7aga ya3ny
train_processeddf = pd.DataFrame(X_train_processed)
test_processeddf = pd.DataFrame(X_test_processed)

print("Data preprocessing complete.")

# Random Forest Model

In [ ]:
rf = RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=4)
rf.fit(X_train_processed, y_train)
rf.score(X_train_processed, y_train)

## 5. What Matters Most? Feature Importances from Our Model
After training our RandomForest model, we can ask it: "Which features did you rely on the most to make your predictions?" This plot summarizes the answer and is one of the most important outputs of our analysis.

In [ ]:
# Get the feature names after preprocessing
# This part is a bit tricky because the preprocessor changes the column order and names
# We need to reconstruct the feature names in the correct order

# Get feature names from the 'one-hot' encoder
onehot_feature_names = preprocessor.named_transformers_['1hot'].get_feature_names_out(nominal_cols)

# Combine all feature names in the correct order
# 1. Ordinal features
# 2. OneHot encoded features
# 3. Scaled numeric features
processed_feature_names = ord_cols + list(onehot_feature_names) + numeric_cols

# Create a pandas Series for feature importances
importances = pd.Series(rf.feature_importances_, index=processed_feature_names)

# Sort the feature importances in descending order
sorted_importances = importances.sort_values(ascending=False)

# Plot the feature importances
plt.figure(figsize=(12, 10))
sns.barplot(x=sorted_importances, y=sorted_importances.index, palette='mako')
plt.title('Feature Importances from RandomForest Model', fontsize=18)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.show()

In [ ]:
# Feature Engineering - Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_processed)
X_test_poly = poly.transform(X_test_processed)

In [ ]:
#rf2 = RandomForestRegressor(n_estimators=5000, random_state=42, n_jobs=4)
#rf2.fit(X_train_poly, y_train)
#rf2.score(X_train_poly, y_train)

In [ ]:
#y_test = rf2.predict(X_test_poly)
y_test = rf.predict(X_test_processed)
submi = sample_submission.copy()

In [ ]:
submi['Y'] = y_test[:len(sample_submission)]

In [ ]:
submi.to_csv('submission.csv', index=False)